# Introduction
This notebook contains key suggestions from Ewald for using ib_insync API

## Getting reliable price data
For large number of contracts, this has been a challenge with ib_insync. Here is [his suggestion](https://groups.io/g/insync/message/3055) using Events, with Asyncio options to get tick data.

In [ ]:
# To be run only once
import asyncio
from ib_insync import *
ib = IB().connect('127.0.0.1', 1300, 0)

util.startLoop()

In [ ]:
def onPendingTickers(pendingTickers):
    global n1
    n1 += sum(len(t.ticks) for t in pendingTickers)
    print(ib.wrapper.lastTime)
    print('A', n1)

async def coro():
    global n2
    while True:
        pendingTickers = await ib.pendingTickersEvent
        n2 += sum(len(t.ticks) for t in pendingTickers)
        print('B', n2)

async def coro2():
    global n3
    async for pendingTickers in ib.pendingTickersEvent:
        n3 += sum(len(t.ticks) for t in pendingTickers)
        print('C', n3)

async def coro3():
    global n4
    while True:
        await ib.updateEvent
        n4 += sum(len(t.ticks) for t in ib.pendingTickers())
        print('D', n4)


n1 = n2 = n3 = n4 = n5 = 0

contracts = [
    Forex('EURUSD'),
    Forex('GBPUSD'),
    Stock('SPY', 'SMART', 'USD'),
    ContFuture("ES", exchange="GLOBEX", currency="USD")]
ib.qualifyContracts(*contracts)
tickers = [ib.reqMktData(c) for c in contracts]
ib.pendingTickersEvent += onPendingTickers
asyncio.ensure_future(coro())
asyncio.ensure_future(coro2())
asyncio.ensure_future(coro3())

In [11]:
tickers

2020-02-18 09:06:54.747925+00:00
A 2247
D 2247
C 2247
B 2247


[Ticker(contract=Forex('EURUSD', conId=12087792, exchange='IDEALPRO', localSymbol='EUR.USD', tradingClass='EUR.USD'), time=datetime.datetime(2020, 2, 18, 9, 6, 54, 455243, tzinfo=datetime.timezone.utc), bid=1.0829, bidSize=7000000, ask=1.08295, askSize=24000000, prevBid=1.08285, prevBidSize=8000000, prevAsk=1.0829, prevAskSize=23500000, high=1.0838, low=1.0823, close=1.0836),
 Ticker(contract=Forex('GBPUSD', conId=12087797, exchange='IDEALPRO', localSymbol='GBP.USD', tradingClass='GBP.USD'), time=datetime.datetime(2020, 2, 18, 9, 6, 54, 747925, tzinfo=datetime.timezone.utc), bid=1.29945, bidSize=5000000, ask=1.2995, askSize=1000000, prevBid=1.2994, prevBidSize=6000000, prevAsk=1.29945, prevAskSize=2000000, high=1.3008, low=1.2971, close=1.3004, ticks=[TickData(time=datetime.datetime(2020, 2, 18, 9, 6, 54, 747925, tzinfo=datetime.timezone.utc), tickType=0, price=1.29945, size=6000000), TickData(time=datetime.datetime(2020, 2, 18, 9, 6, 54, 747925, tzinfo=datetime.timezone.utc), tickType

In [ ]:
# This should be avoided
while ib.waitOnUpdate():
    # this will miss ticks
    n5 += sum(len(t.ticks) for t in tickers)
    print('E', n5)

In [ ]:
tickers